In [61]:
import os

In [62]:
os.chdir(r"C:\Users\Herold-J\Documents\GitHub\MacroHistoryProjects")
print("Directory changed") 

Directory changed


In [63]:
# Import some necessary modules and packages

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests


In [64]:
# Download always the newest version to directory

url = "http://www.macrohistory.net/JST/JSTdatasetR4.xlsx"
r   = requests.get(url, allow_redirects= True)

output = open('JSTdatasetR4.xlsx', 'wb')
output.write(r.content)
output.close()

# Import data
df = pd.read_excel("JSTdatasetR4.xlsx",sheet_name = "Data")

In [ ]:
# Start data preparation

country_list = df["country"].unique()

var_list     = ["cpi","money", "gdp", "thh","tbus","hpnom"]
grouped      = df.groupby("country")[var_list]

new_vars     = ["inflation","money_growth","gdp_growth","loanHH_growth","loanBus_growth","HPI_growth"]

df[new_vars] = (grouped.diff(1)/grouped.shift(1))*100

def windsorize(col_name,lb=1,ub=99):
    df_out = df[col_name].copy()
    bounds = [np.percentile(df_out.dropna(),i) for i in [lb,ub]]
    df_out[(df_out < bounds[0]) | (df_out > bounds[1])] = np.nan
    return df_out

for item in new_vars:
    df[item] = windsorize(item)
    
df["money_multiplier1"] = df["loanHH_growth"]/df["money_growth"]    
df["money_multiplier2"] = df["loanBus_growth"]/df["money_growth"]    

df["money_multiplier1"] = windsorize("money_multiplier1")
df["money_multiplier2"] = windsorize("money_multiplier2")

#Select period 
df = df.loc[np.logical_and(df.year >= 1948,df.year <= 2016)]


# Plots

sns.lmplot(x = "money_growth", y = "inflation", data = df, hue = "country",fit_reg= False)
sns.regplot(x = "money_growth", y= "inflation", data= df, scatter= False)

sns.lmplot(x = "HPI_growth", y = "inflation", data = df, hue = "country",fit_reg= False)
sns.regplot(x = "HPI_growth", y= "inflation", data= df, scatter= False)

sns.lmplot(x = "loanHH_growth", y = "inflation", data = df, hue = "country",fit_reg= False)
sns.regplot(x = "loanHH_growth", y= "inflation", data= df, scatter= False)

sns.lmplot(x = "loanBus_growth", y = "inflation", data = df, hue = "country",fit_reg= False)
sns.regplot(x = "loanBus_growth", y= "inflation", data= df, scatter= False)